In [1]:
import pandas as pd
import os
import numpy as np
import numpy as np
import pandas as pd
from focal_loss import BinaryFocalLoss
import shap
from datetime import datetime
import warnings
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, recall_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from scipy.stats import norm, multivariate_normal
from scipy.stats import mannwhitneyu
from scipy.spatial import cKDTree as KDTree
from statsmodels.stats.contingency_tables import mcnemar 

from adapt.feature_based import DANN, ADDA, WDGRL, CCSA, CDAN, MCD, MDD
import tensorflow as tf
from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import Input, Dense, Reshape,Layer, Dropout, Activation, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import BinaryFocalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from keras.regularizers import l1,l2
import keras.backend as K

from river.drift import PageHinkley
from river.tree import HoeffdingTreeClassifier, HoeffdingAdaptiveTreeClassifier
from river import stream
from river import compose
from river.datasets import synth
from river import ensemble
from river import linear_model
from river import metrics
from river import neighbors

import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.metrics import geometric_mean_score

from sklearn.cluster import AgglomerativeClustering, KMeans
from numpy.linalg import norm
from sklearn.metrics import silhouette_score
from sklearn.metrics  import pairwise_distances
from sklearn.preprocessing import normalize

from scipy.stats import wilcoxon, kendalltau
from sklearn.metrics import ndcg_score, dcg_score

warnings.filterwarnings("ignore")
pd.options.display.max_columns = 50
colors = ['#1e88e5', '#f52757']  # Pink to Blue
n_bins = 50  # Number of bins in the colormap
shap_colormap = LinearSegmentedColormap.from_list("pink_to_blue", colors, N=n_bins)
plt.rc('axes', axisbelow=True)


2025-02-13 10:19:12.693859: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-13 10:19:13.468362: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
#synthetic dataset generating

"""noisePercentage=0.1
numberOfSamples = 1000
datasets_names = ["SEA_A", "SEA_G", "HYPER_A", "HYPER_G"]
all_synthetic = {}

sea_abrupt = synth.ConceptDriftStream(
                stream=synth.SEA(variant=0),
                drift_stream=synth.SEA(variant=1, noise=noisePercentage),
                position=int(numberOfSamples / 2), width=3)

sea_gradual= synth.ConceptDriftStream(
                stream=synth.SEA(variant=1),
                drift_stream=synth.SEA(variant=2, noise=noisePercentage),
                position=int(numberOfSamples / 2), width=0.1*numberOfSamples)


hyperplane_abrupt = synth.ConceptDriftStream(
                stream=synth.Hyperplane(n_features = 4),
                drift_stream=synth.Hyperplane(n_features = 4, n_drift_features = 2, noise_percentage=noisePercentage),
                position=int(numberOfSamples / 2), width=3)

hyperplane_gradual = synth.ConceptDriftStream(
                stream=synth.Hyperplane(n_features = 4),
                drift_stream=synth.Hyperplane(n_features = 4, n_drift_features =2, noise_percentage=noisePercentage),
                position=int(numberOfSamples / 2), width=0.1*numberOfSamples)

for i, dataset in enumerate([sea_abrupt, sea_gradual, hyperplane_abrupt, hyperplane_gradual]):
    data = []
    labels = []
    
    for x, y in dataset.take(numberOfSamples):
        data.append(list(x.values()))
        labels.append(y)
    
    df = pd.DataFrame(data, columns=list(x.keys()))
    df['class'] = labels
    df.to_csv(f"synthetic/{datasets_names[i]}.csv", index=False)
    all_synthetic[datasets_names[i]] = df"""

'noisePercentage=0.1\nnumberOfSamples = 1000\ndatasets_names = ["SEA_A", "SEA_G", "HYPER_A", "HYPER_G"]\nall_synthetic = {}\n\nsea_abrupt = synth.ConceptDriftStream(\n                stream=synth.SEA(variant=0),\n                drift_stream=synth.SEA(variant=1, noise=noisePercentage),\n                position=int(numberOfSamples / 2), width=3)\n\nsea_gradual= synth.ConceptDriftStream(\n                stream=synth.SEA(variant=1),\n                drift_stream=synth.SEA(variant=2, noise=noisePercentage),\n                position=int(numberOfSamples / 2), width=0.1*numberOfSamples)\n\n\nhyperplane_abrupt = synth.ConceptDriftStream(\n                stream=synth.Hyperplane(n_features = 4),\n                drift_stream=synth.Hyperplane(n_features = 4, n_drift_features = 2, noise_percentage=noisePercentage),\n                position=int(numberOfSamples / 2), width=3)\n\nhyperplane_gradual = synth.ConceptDriftStream(\n                stream=synth.Hyperplane(n_features = 4),\n           

In [3]:
def streaming_learning(X: pd.DataFrame,
                       model_1 = HoeffdingTreeClassifier(),
                       model_2 = HoeffdingAdaptiveTreeClassifier(),
                       model_3 = ensemble.ADWINBaggingClassifier(model=linear_model.LogisticRegression()),
                       model_4 = neighbors.KNNClassifier(),
                       x_cols = ["x", "y"],
                       y_col = ["class"]
                       ):
    all_predictions_HTC = []
    all_predictions_HATC = []
    all_predictions_adwin = []
    all_predictions_knn = []
    x = X[x_cols]
    y = X[y_col]

    model_1 = compose.Pipeline(
    ('tree', model_1))
    model_2 = compose.Pipeline(
    ('tree', model_2))
    model_3 = compose.Pipeline(
    ("adwin", model_3))
    model_4 = compose.Pipeline(
    ("knn", model_4))
    
    for xi, yi in stream.iter_pandas(x, y):

      #classical hoeffding tree classifier  
      y_pred = model_1.predict_one(xi)
      all_predictions_HTC.append(y_pred)
      model_1.learn_one(xi, yi[y_col[0]])

      #ensable hoeffding tree classifier  
      y_pred = model_2.predict_one(xi)
      all_predictions_HATC.append(y_pred)
      model_2.learn_one(xi, yi[y_col[0]])

      #adwin
      y_pred = model_3.predict_one(xi)
      all_predictions_adwin.append(y_pred)
      model_3.learn_one(xi, yi[y_col[0]])

      #knn
      y_pred = model_4.predict_one(xi)
      all_predictions_knn.append(y_pred)
      model_4.learn_one(xi, yi[y_col[0]])
        
    return all_predictions_HTC, all_predictions_HATC, all_predictions_adwin, all_predictions_knn

In [4]:
def stream_domain_adaptation(X:pd.DataFrame,
                             ccsa_enc,
                             ccsa_task,
                             ccsa_disc,
                             all_predictions_stream_HTC,
                             all_predictions_stream_HATC,
                             all_predictions_adwin,
                             all_predictions_knn,    
                             stream_change,
                             batch_size,      
                             stream_col="cluster",
                             x_cols = ["x", "y"],
                             y_col = ["class"]):


  


  print("---------------STREAMING LEARNING---------------")
  #all_predictions_stream, stream_change = streaming_learning(X, delta, x_cols, y_col)

  all_results_da_incremental = pd.DataFrame()
  all_results_da_simple = pd.DataFrame()
  result_info = pd.DataFrame()
  df_pred_da = pd.DataFrame()
  sv_previous = []
    
  split_X = split_dataframes_based_on_drift(X, stream_change)
  data_source = split_X[0]
  print(f"CHANGES: {stream_change}")
  Xs = data_source[x_cols]
  ys = data_source[y_col].values
  training = 0


  stream_change.append(len(X))

  print("--------------- DOMAIN ADAPTATION - BATCHES ---------------")


  for i in range(0, len(split_X)-1):
    
    data_target = split_X[i+1]
    Xt = data_target[x_cols]
    yt = data_target[y_col].values
    start_batch = stream_change[i]
    end_batch = start_batch + batch_size
    next_stream_change = stream_change[i+1]
    batch_training = 0  
    f1_source = 0
    da_retraining = 0

    print(f"ITERATION : {i}")
    print(f"START_BATCH: {start_batch}, END_BATCH: {end_batch}, NEXT_CHANGE: {next_stream_change}")


    while end_batch < next_stream_change:

      Xt_batch = X[x_cols][start_batch:end_batch]
      yt_batch = X[y_col][start_batch:end_batch].values
      EPOCHS = 300
      INIT_LR = 0.001 
      decay=INIT_LR / EPOCHS
      model = CCSA(ccsa_enc, ccsa_task, Xt=Xt_batch, yt=yt_batch, metrics=["acc"], 
               loss=BinaryFocalCrossentropy(gamma=1,  apply_class_balancing=False, alpha=0.25), 
               random_state=0, optimizer=Adam(learning_rate=INIT_LR, clipvalue=0.5))

      model.compile()
      f1_source = 0
      da_retraining = 0


      while (f1_source < 0.5 and (da_retraining<5)):
          model.fit(X=Xs, y=ys, Xt=Xt_batch, yt=yt_batch, epochs=EPOCHS, batch_size=128, verbose=0)
          y_predict_source = model.predict(Xs)[:,0]
          y_predict_source = list(map(lambda x: 1 if x>0.5 else 0, y_predict_source))
          f1_source = f1_score(y_predict_source, ys)
          da_retraining += 1
     
     
      training = training + 1
    
      next_batch = end_batch + batch_size

      if next_batch < next_stream_change:

        

        Xt_next_batch = X[x_cols][end_batch:next_batch]
        yt_next_batch = X[y_col][end_batch:next_batch].values
        y_predict_next_batch = model.predict(Xt_next_batch)[:,0]
        y_predict_next_batch = list(map(lambda x: 1 if x>0.5 else 0, y_predict_next_batch))
        df_pred = pd.DataFrame(data=y_predict_next_batch, columns=["y_pred_da"], index=Xt_next_batch.index)
        df_pred_da = pd.concat([df_pred_da, df_pred])
        y_pred_stream_HTC = all_predictions_stream_HTC[end_batch:next_batch]
        y_pred_stream_HATC = all_predictions_stream_HATC[end_batch:next_batch]
        y_pred_stream_knn = all_predictions_knn[end_batch:next_batch]
        y_pred_stream_adwin =  all_predictions_adwin[end_batch:next_batch]

    

        df_result = pd.DataFrame(data={
                          "data_source" : ['1'],
                          "data_target" : [f"{i+2}"],
                          "start_batch_fit" : [start_batch],
                          "end_batch_fit" : [end_batch],
                          "start_batch_predict" : [end_batch],
                          "end_batch_predict" : [end_batch + len(y_predict_next_batch)],
                          "len batch predicted" :  [len(y_predict_next_batch)],
                          "f1_da" : [f1_score(yt_next_batch, y_predict_next_batch, average='macro')],
                          "f1_HTC" : [f1_score(yt_next_batch, y_pred_stream_HTC, average='macro')],
                          "f1_HATC" : [f1_score(yt_next_batch, y_pred_stream_HATC, average='macro')],
                          "f1_knn" : [f1_score(yt_next_batch, y_pred_stream_knn, average='macro')],
                          "f1_adwin" : [f1_score(yt_next_batch, y_pred_stream_adwin, average='macro')],
                          "G_da" : [geometric_mean_score(yt_next_batch, y_predict_next_batch, average='binary')],
                          "G_HTC" : [geometric_mean_score(yt_next_batch, y_pred_stream_HTC, average='binary')],
                          "G_HATC" : [geometric_mean_score(yt_next_batch, y_pred_stream_HATC, average='binary')],
                          "G_knn" : [geometric_mean_score(yt_next_batch, y_pred_stream_knn, average='binary')],
                          "G_adwin" : [geometric_mean_score(yt_next_batch, y_pred_stream_adwin, average='binary')],
                          })

        result_info = pd.concat([result_info, df_result])

        end_batch = next_batch

      else:
        
        Xt_next_batch = X[x_cols][end_batch:next_stream_change]
        yt_next_batch = X[y_col][end_batch:next_stream_change].values
        y_predict_next_batch= model.predict(Xt_next_batch)[:,0]
       
        y_predict_next_batch = list(map(lambda x: 1 if x>0.5 else 0, y_predict_next_batch))
        df_pred = pd.DataFrame(data=y_predict_next_batch, columns=["y_pred_da"], index=Xt_next_batch.index)
        df_pred_da = pd.concat([df_pred_da, df_pred])
        y_pred_stream_HTC = all_predictions_stream_HTC[end_batch:next_stream_change]
        y_pred_stream_HATC = all_predictions_stream_HATC[end_batch:next_stream_change]
        y_pred_stream_knn = all_predictions_knn[end_batch:next_stream_change]
        y_pred_stream_adwin =  all_predictions_adwin[end_batch:next_stream_change]
          

        df_result = pd.DataFrame(data={
                          "data_source" : ['1'],
                          "data_target" : [f"{i+2}"],
                          "start_batch_fit" : [start_batch],
                          "end_batch_fit" : [end_batch],
                          "start_batch_predict" : [end_batch],
                          "end_batch_predict" : [end_batch + len(y_predict_next_batch)],
                          "len batch predicted" :  [len(y_predict_next_batch)],
                          "f1_da" : [f1_score(yt_next_batch, y_predict_next_batch, average='macro')],
                          "f1_HTC" : [f1_score(yt_next_batch, y_pred_stream_HTC, average='macro')],
                          "f1_HATC" : [f1_score(yt_next_batch, y_pred_stream_HATC, average='macro')],
                          "f1_knn" : [f1_score(yt_next_batch, y_pred_stream_knn, average='macro')],
                          "f1_adwin" : [f1_score(yt_next_batch, y_pred_stream_adwin, average='macro')],
                          "G_da" : [geometric_mean_score(yt_next_batch, y_predict_next_batch, average='binary')],
                          "G_HTC" : [geometric_mean_score(yt_next_batch, y_pred_stream_HTC, average='binary')],
                          "G_HATC" : [geometric_mean_score(yt_next_batch, y_pred_stream_HATC, average='binary')],
                          "G_knn" : [geometric_mean_score(yt_next_batch, y_pred_stream_knn, average='binary')],
                          "G_adwin" : [geometric_mean_score(yt_next_batch, y_pred_stream_adwin, average='binary')]
                          })

        result_info = pd.concat([result_info, df_result])
        break

    
    df_pred_stream_HTC = pd.DataFrame(data=all_predictions_stream_HTC, columns=["y_pred_stream_HTC"], index=X.index)
    df_pred_stream_HATC = pd.DataFrame(data=all_predictions_stream_HATC, columns=["y_pred_stream_HATC"], index=X.index)
    df_pred_stream_knn = pd.DataFrame(data=all_predictions_knn, columns=["y_pred_stream_knn"], index=X.index)
    df_pred_stream_adwin = pd.DataFrame(data=all_predictions_adwin, columns=["y_pred_stream_adwin"], index=X.index)
    df_true = pd.DataFrame(data=X[y_col].values, columns=["y_true"], index=X.index)
    df_da = pd.merge(df_pred_da, df_pred_stream_HTC, left_index=True, right_index=True)
    df_da = pd.merge(df_da, df_pred_stream_HATC, left_index=True, right_index=True)
    df_da = pd.merge(df_da, df_pred_stream_knn, left_index=True, right_index=True)
    df_da = pd.merge(df_da, df_pred_stream_adwin, left_index=True, right_index=True)
    df_stream_da = pd.merge(df_da, df_true, left_index=True, right_index=True)
   


    

  return result_info, df_stream_da

## Test the CCSA model

In [5]:
def split_dataframes_based_on_drift(X: pd.DataFrame,
                                    changes: list):
  list_of_change = changes.copy()
  split_dataframes = []
  start_index = 0
  end_index = 0
  list_of_change.append(len(X))
  if len(changes) < 1:
    return X
  else:
    for i in range(len(list_of_change)):
      end_index = list_of_change[i]
      split_dataframes.append(X[start_index:end_index])
      start_index=end_index
    return split_dataframes

In [6]:
def stats_in_domains(stream_change, df_stream):
    result_info = pd.DataFrame()
    for i in range(len(stream_change)-1):
        start_stream = stream_change[i]
        end_stream = stream_change[i+1]
        analysed_stream = df_stream[df_stream.index>start_stream]
        analysed_stream = analysed_stream[analysed_stream.index<end_stream]
        true = analysed_stream["y_true"]
        pred = analysed_stream["y_pred_da"]
        stats = pd.DataFrame(data={
                              "data_source" : ['1'],
                              "data_target" : [f"{i+2}"],
                              "start_domain" : [start_stream],
                              "end_domain": [end_stream],
                              "len batch" :  [len(analysed_stream)],
                              "f1_da" : [f1_score(true, pred, average='macro')],
                              "G_da" : [geometric_mean_score(true, pred)],
                              "accuracy_da" : [accuracy_score(true, pred)],
                              "precision_da" : [precision_score(true, pred, average='macro')],
                              "recall_da" : [recall_score(true, pred, average='macro')],
                              })
        result_info = pd.concat([result_info, stats])
    result_info = result_info.reset_index(drop=True)
    return result_info

In [7]:
#stream_filepath = f"{filepath}/df_stream_da_monday_v1.csv"
#df_stream = pd.read_csv(stream_filepath, index_col=0)
#change_filepath = f"{filepath}/stream_change_monday_v1.csv"
#stream_change = pd.read_csv(change_filepath, index_col=0)["0"].values
#stats = stats_in_domains(stream_change = stream_change, df_stream = df_stream)

In [8]:
def KLdivergence(x, y):
  """Compute the Kullback-Leibler divergence between two multivariate samples.
  Parameters
  ----------
  x : 2D array (n,d)
    Samples from distribution P, which typically represents the true
    distribution.
  y : 2D array (m,d)
    Samples from distribution Q, which typically represents the approximate
    distribution.
  Returns
  -------
  out : float
    The estimated Kullback-Leibler divergence D(P||Q).
  References
  ----------
  Pérez-Cruz, F. Kullback-Leibler divergence estimation of
continuous distributions IEEE International Symposium on Information
Theory, 2008.
  """


  # Check the dimensions are consistent
  x = np.atleast_2d(x)
  y = np.atleast_2d(y)

  n,d = x.shape
  m,dy = y.shape

  assert(d == dy)


  # Build a KD tree representation of the samples and find the nearest neighbour
  # of each point in x.
  xtree = KDTree(x)
  ytree = KDTree(y)

  # Get the first two nearest neighbours for x, since the closest one is the
  # sample itself.
  r = xtree.query(x, k=2, eps=.01, p=2)[0][:,1]
  s = ytree.query(x, k=1, eps=.01, p=2)[0]

  # There is a mistake in the paper. In Eq. 14, the right side misses a negative sign
  # on the first term of the right hand side.
  log_ = -np.log(r/s)
  log_= log_[np.isfinite(log_)]
    
  return log_.sum() * d / n + np.log(m / (n - 1.))
  #return r, s, -np.log(r/s)

In [9]:
def distribution_drift_detection(X: pd.DataFrame,
                                 referance_samples = 100, 
                                 x_cols = ["x", "y"],
                                 min_instances=10,
                                 samples_to_check = 1,
                                 delta=0.05,
                                 threshold=4,
                                 alpha=0.9999,
                                 save=False
                                 
                                ):

    reference_distribution = X[x_cols][:referance_samples].values 
    X["KL"] = -1
    stream_change = []
    ph = PageHinkley(min_instances=min_instances, delta=delta, threshold=threshold, alpha=alpha)
    start_drift = 2
    referance_indx_start = referance_indx_end =  0
    
    for i in range(referance_samples, len(X)-samples_to_check, samples_to_check):

        sample =  X[x_cols].loc[i:i+samples_to_check].values 
        
        kl_divergence = KLdivergence(reference_distribution, sample)
        X["KL"].loc[i] = kl_divergence
        ph.update(kl_divergence)
        if ph.drift_detected:
          #print("detected drift!")
          stream_change.append(i)
          start_drift = i
          reference_distribution = X[x_cols][start_drift-2:i].values

        else:
            reference_distribution = X[x_cols][start_drift-2:i].values
          

    return stream_change, X

In [10]:
SEA_A = pd.read_csv("synthetic/SEA_A.csv")
SEA_G = pd.read_csv("synthetic/SEA_G.csv")
HYPER_A = pd.read_csv("synthetic/HYPER_A.csv")
HYPER_G = pd.read_csv("synthetic/HYPER_G.csv")

In [11]:
datasets_names = ["SEA_A", "SEA_G", "HYPER_A", "HYPER_G"]
all_synthetic = {}
all_synthetic["SEA_A"] = SEA_A
all_synthetic["SEA_G"] = SEA_G
all_synthetic["HYPER_A"] = HYPER_A
all_synthetic["HYPER_G"] = HYPER_G

In [12]:
batch_size=15
name = "SEA_A"
print(name)
X = all_synthetic[name]
x_cols = X.columns.values[:-1]
print(x_cols)

stream_change_detected, X_det =  distribution_drift_detection(X.copy(),
                                 referance_samples = 50, 
                                 x_cols = x_cols,
                                 min_instances = 1,
                                 samples_to_check = 2,
                                 delta = 0.005,
                                 threshold = 10,
                                 alpha = 0.9999,
                                 save = False
                                 
                                )

def get_encoder(input_shape=(X[x_cols].shape[1],)):
    model = Sequential()
    model.add(Dense(100, activation='relu',
                        input_shape=input_shape))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(2, activation="sigmoid"))
    model.compile(optimizer=Adam(learning_rate=0.00001), loss=BinaryFocalLoss(gamma=10))
    return model

def get_task(input_shape=(X[x_cols].shape[1],)):
    model = Sequential()
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer=Adam(learning_rate=0.00001), loss=BinaryFocalLoss(gamma=10))
    return model
    
def get_discriminator(input_shape=(X[x_cols].shape[1],)):
    model = Sequential()
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer=Adam(learning_rate=0.001), loss=BinaryFocalLoss(gamma=2))
    return model



ccsa_enc = get_encoder()
ccsa_disc= get_discriminator()
ccsa_task = get_task()


all_predictions_HTC, all_predictions_HATC, all_predictions_adwin, all_predictions_knn = streaming_learning(X, 
                                                               model_1 = HoeffdingTreeClassifier(),  
                                                               model_2 = HoeffdingAdaptiveTreeClassifier(), 
                                                               model_3 = ensemble.ADWINBaggingClassifier(model=linear_model.LogisticRegression()),
                                                               model_4 = neighbors.KNNClassifier(),
                                                               x_cols = x_cols, 
                                                               y_col = ["class"])


result_batches, predictions = stream_domain_adaptation(X=X, 
                                                 ccsa_enc=ccsa_enc, 
                                                 ccsa_task=ccsa_task, 
                                                 ccsa_disc=ccsa_disc, 
                                                 all_predictions_stream_HTC=all_predictions_HTC, 
                                                 all_predictions_stream_HATC=all_predictions_HATC, 
                                                 all_predictions_adwin=all_predictions_adwin, 
                                                 all_predictions_knn=all_predictions_knn, 
                                                 stream_change=stream_change_detected, 
                                                 batch_size = batch_size, 
                                                 x_cols = x_cols,
                                                 y_col = ["class"])

SEA_A
['0' '1' '2']


2025-02-13 10:19:15.915035: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-02-13 10:19:15.915064: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: 002bfeb9166b
2025-02-13 10:19:15.915071: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: 002bfeb9166b
2025-02-13 10:19:15.915318: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 550.90.7
2025-02-13 10:19:15.915339: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 550.90.7
2025-02-13 10:19:15.915344: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 550.90.7


---------------STREAMING LEARNING---------------
CHANGES: [230, 302, 358, 416, 444, 578, 654, 816]
--------------- DOMAIN ADAPTATION - BATCHES ---------------
ITERATION : 0
START_BATCH: 230, END_BATCH: 245, NEXT_CHANGE: 302
ITERATION : 1
START_BATCH: 302, END_BATCH: 317, NEXT_CHANGE: 358
ITERATION : 2
START_BATCH: 358, END_BATCH: 373, NEXT_CHANGE: 416
ITERATION : 3
START_BATCH: 416, END_BATCH: 431, NEXT_CHANGE: 444
ITERATION : 4
START_BATCH: 444, END_BATCH: 459, NEXT_CHANGE: 578
ITERATION : 5
START_BATCH: 578, END_BATCH: 593, NEXT_CHANGE: 654
ITERATION : 6
START_BATCH: 654, END_BATCH: 669, NEXT_CHANGE: 816
ITERATION : 7
START_BATCH: 816, END_BATCH: 831, NEXT_CHANGE: 1000


In [13]:
df_res = predictions.replace({True: 1, False: 0})
scores = {"f1_DA" : [f1_score(df_res["y_true"], df_res["y_pred_da"])],
          "f1_HTC" : [f1_score(df_res["y_true"], df_res["y_pred_stream_HTC"])],
          "f1_HATC" : [f1_score(df_res["y_true"], df_res["y_pred_stream_HATC"])],
          "f1_knn" : [f1_score(df_res["y_true"], df_res["y_pred_stream_knn"])],
          "f1_adwin" : [f1_score(df_res["y_true"], df_res["y_pred_stream_adwin"])],
          "G_da" : [geometric_mean_score(df_res["y_true"], df_res["y_pred_da"], average='binary')],
          "G_HTC" : [geometric_mean_score(df_res["y_true"], df_res["y_pred_stream_HTC"], average='binary')],         
          "G_HATC" : [geometric_mean_score(df_res["y_true"], df_res["y_pred_stream_HATC"], average='binary')],
          "G_knn" : [geometric_mean_score(df_res["y_true"], df_res["y_pred_stream_knn"], average='binary')],
          "G_adwin" : [geometric_mean_score(df_res["y_true"], df_res["y_pred_stream_adwin"], average='binary')],
          "accuracy_da" : [accuracy_score(df_res["y_true"], df_res["y_pred_da"])],
          "accuracy_HTC" : [accuracy_score(df_res["y_true"], df_res["y_pred_stream_HTC"])],         
          "accuracy_HATC" : [accuracy_score(df_res["y_true"], df_res["y_pred_stream_HATC"])],
          "accuracy_knn" : [accuracy_score(df_res["y_true"], df_res["y_pred_stream_knn"])],
          "accuracy_adwin" : [accuracy_score(df_res["y_true"], df_res["y_pred_stream_adwin"])],
                         }
display(pd.DataFrame(scores))

,f1_DA,f1_HTC,f1_HATC,f1_knn,f1_adwin,G_da,G_HTC,G_HATC,G_knn,G_adwin,accuracy_da,accuracy_HTC,accuracy_HATC,accuracy_knn,accuracy_adwin
0,0.916968,0.895662,0.898007,0.903226,0.80947,0.873586,0.827535,0.830929,0.850336,0.662301,0.893846,0.863077,0.866154,0.875385,0.74
